####import libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
import pathlib
import os
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

####Download Dataset

In [ ]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos.tar', origin=dataset_url, extract=True)
data_dir = pathlib.Path(data_dir).with_suffix('')

In [3]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print('Number of images = ', image_count)

Number of images =  3670


In [4]:
daisy = list(data_dir.glob('daisy/*'))            #633
dandelion = list(data_dir.glob('dandelion/*'))    #898
roses = list(data_dir.glob('roses/*'))            #641
sunflowers = list(data_dir.glob('sunflowers/*'))  #699
tulips = list(data_dir.glob('tulips/*'))          #799

#PIL.Image.open(daisy[0])

####Load data

In [5]:
#some parameters
batch_size = 32
img_height = 180
img_width = 180

train_ds = tf.keras.utils.image_dataset_from_directory(data_dir, validation_split=0.2, subset="training", seed=123,
                                                       image_size=(img_height, img_width), batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(data_dir, validation_split=0.2, subset="validation", seed=123,
                                                     image_size=(img_height, img_width), batch_size=batch_size)

class_names = train_ds.class_names
# print(class_names)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.


####Visualize the data

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 180, 180, 3)
(32,)


####Configure the dataset for performance

In [6]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

####Data augmentation

In [7]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal", input_shape=(img_height, img_width,3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
    ]
)

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")

####Define Model

In [9]:
num_classes = len(class_names)

model = Sequential([
  data_augmentation,
  layers.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='elu'),
  layers.Conv2D(32, 3, padding='same', activation='elu'),
  layers.MaxPool2D(pool_size=(2, 2)),
  layers.BatchNormalization(axis=-1),

  layers.Conv2D(64, 3, padding='same', activation='elu'),
  layers.Conv2D(128, 3, padding='same', activation='elu'),
  layers.MaxPool2D(pool_size=(2, 2)),
  keras.layers.BatchNormalization(axis=-1),

  layers.Flatten(),
  layers.Dense(512, activation='elu'),
  layers.BatchNormalization(),
  layers.Dropout(0.5),
  layers.Dense(num_classes, name="outputs")
])

####Compile and save model

In [10]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
#model.summary()

In [ ]:
epochs=5
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/5
92/92 [==============================] - 35s 148ms/step - loss: 2.0263 - accuracy: 0.4275 - val_loss: 1.6380 - val_accuracy: 0.3924
Epoch 2/5
92/92 [==============================] - 11s 114ms/step - loss: 1.3963 - accuracy: 0.5112 - val_loss: 1.6246 - val_accuracy: 0.3665
Epoch 3/5
92/92 [==============================] - 11s 115ms/step - loss: 1.2435 - accuracy: 0.5480 - val_loss: 1.2788 - val_accuracy: 0.4578
Epoch 4/5
92/92 [==============================] - 11s 117ms/step - loss: 1.1293 - accuracy: 0.5770 - val_loss: 1.1670 - val_accuracy: 0.5490
Epoch 5/5
92/92 [==============================] - 11s 118ms/step - loss: 1.0577 - accuracy: 0.6063 - val_loss: 0.9862 - val_accuracy: 0.6090


####Visualize training results

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

####Saving and Loading

In [ ]:
model.save("model.h5")

In [ ]:
filepath = '/content/model.h5'
model1 = tf.keras.models.load_model(
    filepath, custom_objects=None, compile=True, options=None
    )

####Predict on new data

In [ ]:
sunflower_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/592px-Red_sunflower.jpg"
sunflower_path = tf.keras.utils.get_file('Red_sunflower', origin=sunflower_url)

img = tf.keras.utils.load_img(
    sunflower_path, target_size=(img_height, img_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model1.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

1/1 [==============================] - 1s 551ms/step
This image most likely belongs to sunflowers with a 96.17 percent confidence.
